#kết nối driver


In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#tải các thư viện cần thiết

In [19]:
!pip install emoji

In [20]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#khai báo thư viện

In [65]:
import pandas as pd
import numpy as np
import re
import nltk
import emoji
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout, Convolution1D, MaxPooling1D, Flatten
from nltk.stem import PorterStemmer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import History
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Embedding, concatenate
from sklearn.metrics import accuracy_score, f1_score, recall_score
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

#đọc dữ liệu từ file csv

In [22]:
# Đọc dữ liệu từ file CSV
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/demo/Reviews.csv')

In [23]:
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


#tiền xử lý dữ liệu

In [24]:
# Loại bỏ các ký tự đặc biệt
data['Text'] = data['Text'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', str(x)))

In [25]:
# Chuyển về chữ thường
data['Text'] = data['Text'].apply(lambda x: x.lower())

In [26]:
# Loại bỏ stopwords
stop_words = set(stopwords.words('english'))
data['Text'] = data['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

In [27]:
# Stemming
stemmer = PorterStemmer()
data['Text'] = data['Text'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['Text'], data['Score'], test_size=0.2, random_state=42)

In [29]:
max_features = 5000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

max_len = 100
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [30]:
#Chuyển đổi nhãn thành one-hot encoding:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

#Xây dựng và huấn luyện mô hình

In [31]:
# Xây dựng mô hình CNN-LSTM
def build_cnn_lstm_model(max_features, max_len):
    model = Sequential()
    model.add(Embedding(max_features, 128, input_length=max_len))
    model.add(Conv1D(64, 3, padding='same', activation='relu'))
    model.add(MaxPooling1D())
    model.add(LSTM(64))
    model.add(Dense(1, activation='linear'))
    return model

# Compile mô hình
cnn_lstm_model = build_cnn_lstm_model(max_features, max_len)
cnn_lstm_model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Huấn luyện mô hình
history_cnn_lstm = cnn_lstm_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=32)


Epoch 1/5
14212/14212 [==============================] - 845s 59ms/step - loss: 0.1389 - accuracy: 0.8333 - val_loss: 0.1389 - val_accuracy: 0.8333
Epoch 2/5
14212/14212 [==============================] - 867s 61ms/step - loss: 0.1389 - accuracy: 0.8333 - val_loss: 0.1389 - val_accuracy: 0.8333
Epoch 3/5
14212/14212 [==============================] - 868s 61ms/step - loss: 0.1389 - accuracy: 0.8333 - val_loss: 0.1389 - val_accuracy: 0.8333
Epoch 4/5
14212/14212 [==============================] - 862s 61ms/step - loss: 0.1389 - accuracy: 0.8333 - val_loss: 0.1389 - val_accuracy: 0.8333
Epoch 5/5
14212/14212 [==============================] - 863s 61ms/step - loss: 0.1389 - accuracy: 0.8333 - val_loss: 0.1389 - val_accuracy: 0.8333


In [32]:

# Xây dựng mô hình Multi-CNN-LSTM
def build_multi_cnn_lstm_model(max_features, max_len):
    # CNN branch
    cnn_branch = Sequential()
    cnn_branch.add(Embedding(max_features, 128, input_length=max_len))
    cnn_branch.add(Conv1D(64, 3, padding='same', activation='relu'))
    cnn_branch.add(MaxPooling1D())
    cnn_branch.add(Flatten())

    # LSTM branch
    lstm_branch = Sequential()
    lstm_branch.add(Embedding(max_features, 128, input_length=max_len))
    lstm_branch.add(LSTM(64))

    # Merge branches
    merged = concatenate([cnn_branch.output, lstm_branch.output])
    merged = Dense(1, activation='linear')(merged)

    model = Model(inputs=[cnn_branch.input, lstm_branch.input], outputs=merged)
    return model

# Compile mô hình
multi_cnn_lstm_model = build_multi_cnn_lstm_model(max_features, max_len)
multi_cnn_lstm_model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Huấn luyện mô hình
history_multi_cnn_lstm = multi_cnn_lstm_model.fit([X_train, X_train], y_train, validation_data=([X_test, X_test], y_test), epochs=5, batch_size=32)


Epoch 1/5
14212/14212 [==============================] - 1554s 109ms/step - loss: 0.1389 - accuracy: 0.8333 - val_loss: 0.1389 - val_accuracy: 0.8333
Epoch 2/5
14212/14212 [==============================] - 1605s 113ms/step - loss: 0.1389 - accuracy: 0.8333 - val_loss: 0.1389 - val_accuracy: 0.8333
Epoch 3/5
14212/14212 [==============================] - 1598s 112ms/step - loss: 0.1389 - accuracy: 0.8333 - val_loss: 0.1389 - val_accuracy: 0.8333
Epoch 4/5
14212/14212 [==============================] - 1608s 113ms/step - loss: 0.1389 - accuracy: 0.8333 - val_loss: 0.1389 - val_accuracy: 0.8333
Epoch 5/5
14212/14212 [==============================] - 1614s 114ms/step - loss: 0.1389 - accuracy: 0.8333 - val_loss: 0.1389 - val_accuracy: 0.8333


#lưu mô hình

In [34]:
cnn_lstm_model.save('food_sentiment_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [35]:
multi_cnn_lstm_model.save('food_sentiment_model2.h5')


#đánh giá mô hình

In [37]:
#moadel_cnn_lstm
loss, accuracy = cnn_lstm_model.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)


3553/3553 [==============================] - 79s 22ms/step - loss: 0.1389 - accuracy: 0.8333
Test loss: 0.1388859897851944
Test accuracy: 0.8333195447921753


In [38]:
#model multi_cnn_lstm
loss, accuracy = multi_cnn_lstm_model.evaluate([X_test, X_test], y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)


3553/3553 [==============================] - 98s 28ms/step - loss: 0.1389 - accuracy: 0.8333
Test loss: 0.1388859897851944
Test accuracy: 0.8333195447921753


In [39]:
# Chia tập huấn luyện thành tập huấn luyện và tập validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# In số lượng của mỗi tập
print("Số lượng mẫu trong tập train:", len(X_train))
print("Số lượng mẫu trong tập test:", len(X_test))
print("Số lượng mẫu trong tập validation:", len(X_val))

Số lượng mẫu trong tập train: 363810
Số lượng mẫu trong tập test: 113691
Số lượng mẫu trong tập validation: 90953


#dự đoán và demo

In [72]:
def predict_sentiment(review):
    # Tiền xử lý dữ liệu cho review nhập vào
    review = re.sub(r'[^a-zA-Z\s]', '', str(review))  # Loại bỏ ký tự đặc biệt
    review = review.lower()  # Chuyển về chữ thường
    review = ' '.join([word for word in review.split() if word not in stop_words])  # Loại bỏ stopwords
    review = ' '.join([stemmer.stem(word) for word in review.split()])  # Stemming
    review_seq = tokenizer.texts_to_sequences([review])  # Chuyển đổi thành sequence
    review_pad = pad_sequences(review_seq, maxlen=max_len)  # Padding để có cùng kích thước với input của mô hình

    # Dự đoán cảm xúc
    sentiment = model.predict(review_pad)[0][0]
    return sentiment



In [73]:
# Chọn một hàng ngẫu nhiên từ tập dữ liệu
random_row_index = np.random.randint(0, len(data))
random_row = data.iloc[random_row_index]

# In thông tin về sản phẩm, đánh giá, và phân loại tương ứng
print("Sản phẩm:", random_row['ProductId'])
print("Đánh giá Text:", random_row['Text'])
print("Điểm đánh giá:", random_row['Score'])
print("Phân loại:", classify_review_with_score(random_row['Score']))


Sản phẩm: B0009K77O8
Đánh giá Text: I love this tea!  The addition of the brown rice kind of takes the edge off the green tea and gives it a very mild and subtle nutty flavor.
Điểm đánh giá: 5
Phân loại: Tích cực (Điểm 5)


In [74]:
# Load pre-trained model
model = load_model('food_sentiment_model.h5')


In [77]:
def predict_sentiment(model, review):
    # Tiền xử lý dữ liệu cho review nhập vào
    review = re.sub(r'[^a-zA-Z\s]', '', str(review))  # Loại bỏ ký tự đặc biệt
    review = review.lower()  # Chuyển về chữ thường
    review = ' '.join([word for word in review.split() if word not in stop_words])  # Loại bỏ stopwords
    review = ' '.join([stemmer.stem(word) for word in review.split()])  # Stemming
    review_seq = tokenizer.texts_to_sequences([review])  # Chuyển đổi thành sequence
    review_pad = pad_sequences(review_seq, maxlen=max_len)  # Padding để có cùng kích thước với input của mô hình

    # Dự đoán cảm xúc
    sentiment = model.predict(review_pad)[0][0]
    return sentiment

In [82]:
while True:
    product_name = input("Nhập tên sản phẩm (hoặc 'q' để thoát): ")
    if product_name.lower() == 'q':
        break

    # Tìm các đánh giá liên quan đến sản phẩm
    relevant_reviews = data[data['ProductId'] == product_name]

    if len(relevant_reviews) == 0:
        print("Không có đánh giá nào cho sản phẩm này.")
        continue

    # Dự đoán và in ra phân loại của các đánh giá
    for _, review_row in relevant_reviews.iterrows():
        sentiment = predict_sentiment(model, review_row['Text'])  # Thêm model vào hàm predict_sentiment
        print("Đánh giá:", review_row['Text'])
        print("Điểm đánh giá:", review_row['Score'])
        if sentiment >= 0.7:
            print("Phân loại: Đánh giá tích cực")
        elif sentiment <= 0.3:
            print("Phân loại: Đánh giá tiêu cực")
        else:
            print("Phân loại: Đánh giá trung tính")
        print()


Nhập tên sản phẩm (hoặc 'q' để thoát): trewtre
Không có đánh giá nào cho sản phẩm này.
Nhập tên sản phẩm (hoặc 'q' để thoát): B001E4KFG0
1/1 [==============================] - 0s 37ms/step
Đánh giá: I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.
Điểm đánh giá: 5
Phân loại: Đánh giá tiêu cực

Nhập tên sản phẩm (hoặc 'q' để thoát): q
